# Практика по кластеризации

In [ ]:
import os
import sys

In [ ]:
sys.path.append(os.path.join('..', '..'))

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

from abc import ABCMeta
from collections import Counter
from IPython.display import display
from functools import lru_cache
from ipywidgets import interact, fixed, IntSlider, FloatSlider
from matplotlib import rcParams
from sklearn.base import TransformerMixin
from sklearn.cluster import (MeanShift, AgglomerativeClustering, DBSCAN,
                             MiniBatchKMeans, KMeans, 
                             SpectralClustering)
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from typing import List

from src.utils.common import get_data_folder, timeit
from src.utils.plots import plot_dendrogram, plot_sorted_nn_dists

%matplotlib inline
rcParams['font.size'] = 14
rcParams['figure.figsize'] = 9, 8

warnings.filterwarnings('ignore')

SEED = 5
np.random.seed(SEED)

### Используемые данные.
Проточная цитометрия — метод исследования дисперсных сред в режиме поштучного анализа элементов дисперсной фазы по сигналам светорассеяния и флуоресценции. Название метода связано с основным приложением, а именно, с исследованием одиночных биологических клеток в потоке.
<img src="../../misc/cytometry.png" width="680"/>

In [ ]:
data_folder = get_data_folder()
dfs = [pd.read_csv(os.path.join(data_folder, 'flowcytometry', file_name)) 
       for file_name in os.listdir(os.path.join(data_folder, 'flowcytometry'))]

In [ ]:
dfs[0].describe()

In [ ]:
for ind, df in enumerate(dfs):
    print(f'Patient {ind + 1}:', df.isnull().any().sum())

In [ ]:
cols = dfs[0].columns

In [ ]:
def scatterplot2d(df, col1='FSC-A-', col2='SSC-A-', 
                  labels=None, 
                  dots_size=6, palette='coolwarm'):
    fig, _ = plt.subplots()
    sns.scatterplot(df[col1], df[col2], hue=labels, s=dots_size, palette=palette)
    fig.canvas.draw()

In [ ]:
scatterplot2d(dfs[0], cols[1], cols[2])

In [ ]:
# избавимся от части выбросов
for i, df in enumerate(dfs):
    mask = (df['FSC-A-'] > 200000) | (df['SSC-A-'] > 240000)
    dfs[i] = df.drop(df[mask].index)

In [ ]:
scatterplot2d(dfs[0], cols[1], cols[2])

### Основные алгоритмы

In [ ]:
@timeit
def clust_and_viz(df, clust_cols, clusterer, dots_size=5):
    clusterer.fit(df[clust_cols])
    labels = clusterer.labels_ if hasattr(clusterer, 'labels_') else clusterer.predict(df[clust_cols])
    print(f'Число кластеров: {len(set(labels))}') 
    scatterplot2d(df, 
                  col1=clust_cols[0], 
                  col2=clust_cols[1], 
                  labels=labels, 
                  dots_size=dots_size)
    return labels

In [ ]:
df = dfs[0].copy()
df_scaled = df.copy()

scaler = StandardScaler()
df_scaled[df_scaled.columns[1:]] = scaler.fit_transform(df_scaled[df_scaled.columns[1:]])

In [ ]:
clust_cols = ['FSC-A-', 'SSC-A-']

#### Meanshift

docs: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html

code: https://github.com/scikit-learn/scikit-learn/blob/95119c13a/sklearn/cluster/_mean_shift.py#L243

* Основной цикл со сдвигом сидов\кернелов\опорных точек (400 строчка): https://github.com/scikit-learn/scikit-learn/blob/95119c13af77c76e150b753485c662b7c52a41a2/sklearn/cluster/_mean_shift.py#L90
* Строчки 422-435 -- Выбираем центры кластеров
* Строчки 438-447 -- Расставляем метки кластеров

In [ ]:
base_meanshift = MeanShift(bandwidth=None, 
                           min_bin_freq=1, 
                           cluster_all=True, 
                           bin_seeding=True, 
                           n_jobs=4)

base_meanshift_labels = clust_and_viz(df_scaled, clust_cols, base_meanshift)

In [ ]:
opt_meanshift = MeanShift(bandwidth=0.7, 
                          min_bin_freq=3,  
                          cluster_all=False, 
                          bin_seeding=True, 
                          n_jobs=4)

opt_meanshift_labels = clust_and_viz(df_scaled, clust_cols, opt_meanshift)

#### Aglomerative clustering

docs: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html

code: https://github.com/scikit-learn/scikit-learn/blob/95119c13a/sklearn/cluster/_agglomerative.py#L681

* При ближайшем рассмотрении выясняется, что сам по себе алгоритм подтягивается из scipy.cluster.hierarchy:

    ** ward_tree: https://github.com/scikit-learn/scikit-learn/blob/95119c13af77c76e150b753485c662b7c52a41a2/sklearn/cluster/_agglomerative.py#L138  (строка 236)
    
    ** https://github.com/scipy/scipy/blob/5ab7426247900db9de856e790b8bea1bd71aec49/scipy/cluster/hierarchy.py#L738
    
    ** https://github.com/scipy/scipy/blob/5ab7426247900db9de856e790b8bea1bd71aec49/scipy/cluster/hierarchy.py#L837
    
    ** https://github.com/scipy/scipy/blob/5ab7426247900db9de856e790b8bea1bd71aec49/scipy/cluster/_hierarchy.pyx#L908
    
    ** вычисление метрик связи: https://github.com/scipy/scipy/blob/5ab7426247900db9de856e790b8bea1bd71aec49/scipy/cluster/_hierarchy_distance_update.pxi

* Своя версия есть только в контексте работы с матрицей связи (connectivity matrix) -- это про накидывание локальной структуры связи между данными

In [ ]:
base_agglomerative = AgglomerativeClustering(n_clusters=2,  # может быть None
                                             affinity='euclidean', 
                                             linkage='ward')

base_agglomerative_labels = clust_and_viz(df_scaled, clust_cols, base_agglomerative)

In [ ]:
full_agglomerative = AgglomerativeClustering(n_clusters=None,
                                             distance_threshold=0, 
                                             affinity='euclidean', 
                                             linkage='average')
full_agglomerative.fit(df_scaled[clust_cols])

plot_dendrogram(full_agglomerative, truncate_mode='level', p=4)

In [ ]:
opt_agglomerative = AgglomerativeClustering(n_clusters=15,
                                            affinity='euclidean', 
                                            linkage='average')

opt_agglomerative_labels = clust_and_viz(df_scaled, clust_cols, opt_agglomerative)

#### DBSCAN

docs: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

code: https://github.com/scikit-learn/scikit-learn/blob/95119c13a/sklearn/cluster/_dbscan.py#L148

code (cython part): https://github.com/scikit-learn/scikit-learn/blob/95119c13af77c76e150b753485c662b7c52a41a2/sklearn/cluster/_dbscan_inner.pyx

In [ ]:
plot_sorted_nn_dists(df_scaled[clust_cols], min_pts=4)

In [ ]:
base_dbscan = DBSCAN(eps=0.5, 
                     min_samples=4, 
                     metric='euclidean', 
                     n_jobs=4)

base_dbscan_labels = clust_and_viz(df_scaled, clust_cols, base_dbscan)

In [ ]:
opt_dbscan = DBSCAN(eps=0.04, 
                    min_samples=4, 
                    metric='euclidean', 
                    n_jobs=4)

opt_dbscan_labels = clust_and_viz(df_scaled, clust_cols, opt_dbscan)

In [ ]:
print(Counter(opt_dbscan_labels))

In [ ]:
scatterplot2d(df_scaled, clust_cols[0], clust_cols[1], labels=opt_dbscan_labels==1)

#### Gaussian Mixture Model (Expectation Maximization)

docs: https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html

code: https://github.com/scikit-learn/scikit-learn/blob/95119c13a/sklearn/mixture/_gaussian_mixture.py#L434

code (BaseMixture class): https://github.com/scikit-learn/scikit-learn/blob/95119c13af77c76e150b753485c662b7c52a41a2/sklearn/mixture/_base.py#L484

fit-метод вызывается из BaseMixture. В GMM лишь определяется m-шаг, выполняющий пересчет параметров распределения (нормального, в данном случае)

In [ ]:
base_gmm = GaussianMixture(n_components=1, 
                           covariance_type='full',
                           n_init=1, 
                           init_params='kmeans', 
                           random_state=SEED)

base_gmm_labels = clust_and_viz(df_scaled, clust_cols, base_gmm)

In [ ]:
opt_gmm = GaussianMixture(n_components=4, 
                          covariance_type='full',
                          n_init=5, 
                          init_params='kmeans', 
                          random_state=SEED)

opt_gmm_labels = clust_and_viz(df_scaled, clust_cols, opt_gmm)

#### Spectral clustering

docs: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html

code: https://github.com/scikit-learn/scikit-learn/blob/95119c13a/sklearn/cluster/_spectral.py#L287
* Вычисляем матрицу связи и вызываем метод spectral_clustering: https://github.com/scikit-learn/scikit-learn/blob/95119c13af77c76e150b753485c662b7c52a41a2/sklearn/cluster/_spectral.py#L162

* Внутри которого строится спектральный эмбеддинг (https://github.com/scikit-learn/scikit-learn/blob/95119c13af77c76e150b753485c662b7c52a41a2/sklearn/manifold/_spectral_embedding.py#L145) 

    ** Лапласиан строится функцией laplacian из scipy.sparse.csgraph: https://github.com/scipy/scipy/blob/v1.6.1/scipy/sparse/csgraph/_laplacian.py#L16-L79

* и запускается метод кластеризации (kmeans по дефолту)

In [ ]:
base_spectral = SpectralClustering(n_clusters=8, 
                                   # n_components=8, 
                                   random_state=SEED, 
                                   n_init=10, 
                                   affinity='nearest_neighbors', 
                                   n_neighbors=10, 
                                   assign_labels='kmeans', 
                                   n_jobs=4)

base_spectral_labels = clust_and_viz(df_scaled, clust_cols, base_spectral)

In [ ]:
opt_spectral = SpectralClustering(n_clusters=5, 
                                  n_components=2, 
                                  random_state=SEED, 
                                  n_init=5, 
                                  affinity='nearest_neighbors', 
                                  n_neighbors=25, 
                                  assign_labels='kmeans', 
                                  n_jobs=4)

opt_spectral_labels = clust_and_viz(df_scaled, clust_cols, opt_spectral)

### Метрики

In [ ]:
# TODO

### Бонусная секция

Создадим словарик с рассмотренными методами (и k-means'ом, куда ж без него)
и ограничениями на их основные параметры

In [ ]:
clustering = {
    'meanshift': {'method': MeanShift, 
                  'params_range': {'bandwidth': [1] + list(np.arange(0.3, 1.5, 0.05)) + [None], 
                                   'bin_seeding': [True, False], 
                                   'n_jobs': [*range(1, 5), -1]}
                 }, 
    'agglomerative': {'method': AgglomerativeClustering, 
                      'params_range': {'n_clusters': [*range(2, 50)], 
                                       'affinity': ['euclidean', 'manhattan'], 
                                       'linkage': ['ward', 'complete', 'average', 'single']}}, 
    'dbscan': {'method': DBSCAN, 
               'params_range': {'eps': [*np.arange(0.01, 0.2, 0.01)], 
                                'min_samples': [*range(1, 26)], 
                                'metric': ['euclidean', 'manhattan'],
                                'n_jobs': [*range(1, 5), -1]}},
    'em': {'method': GaussianMixture, 
           'params_range': {'n_components': [*range(2, 50)], 
                            'covariance_type': ['full', 'tied', 'diag', 'spherical'],
                            'n_init': [*range(1, 6)],
                            'init_params': ['kmeans', 'random'],
                            'random_state': fixed(SEED)}}, 
    'kmeans': {'method': KMeans, 
               'params_range': {'n_clusters': [*range(2, 50)],
                                'n_init': [*range(5, 25)],
                                'random_state': fixed(SEED), 
                                'n_jobs': [*range(1, 5), -1]}},
    'mbkmeans': {'method': MiniBatchKMeans, 
                 'params_range': {'n_clusters': [*range(2, 50)], 
                                  'batch_size': [*range(100, 1001, 100)],
                                  'n_init': [*range(3, 8)],
                                  'random_state': fixed(SEED)}},
    'spectral': {'method': SpectralClustering, 
                 'params_range': {'n_clusters': [*range(2, 50)], 
                                  'n_components': [*range(2, 50)],
                                  'affinity': ['nearest_neighbors'], 
                                  'gamma': [*np.arange(0.5, 2, 0.1)],
                                  'n_neighbors': [*range(2, 25)],                         
                                  'assign_labels': ['kmeans', 'discretize'], 
                                  'n_init': [*range(10, 25)],
                                  'random_state': fixed(SEED), 
                                  'n_jobs': [*range(1, 5), -1]}}
}

Запилим класс, который будет кластеризовать данные и отрисовывать результаты в зависимости от поданных в его метод analysis2d параметров.

Кэшируем результаты фит-предикта, чтобы не пересчитывать все заново, если изменим размер точек на графике

In [ ]:
class InteractiveClusterer:
    def __init__(self, 
                 method: str, 
                 params_range: dict, 
                 dfs: List[pd.DataFrame], 
                 scaler: TransformerMixin = MinMaxScaler()):
        self.method = method
        self.clusterer = None
        self.params_range = params_range
        self.dfs = dfs
        self.curr_df = None
        
    @lru_cache(maxsize=None)
    def fit_predict(self, 
                    patient=0, 
                    col1='FSC-A-', 
                    col2='SSC-A-', 
                    do_scaling=False, 
                    **kwargs): 
        # параметры вне kwargs нужны для кэширования результатов
        self.clusterer = self.method(**kwargs)
        self.clusterer.fit(self.curr_df)
        
        if not isinstance(self.method, ABCMeta):
            return self.clusterer.labels_  
        else:
            # для случая GMM
            return self.clusterer.predict(self.curr_df)
    
    def analysis2d(self, 
                   print_clust_num=False, 
                   dots_size=5, 
                   palette='coolwarm', 
                   patient=0, 
                   col1='FSC-A-', 
                   col2='SSC-A-', 
                   do_scaling=True, 
                   plot_scaled=True, 
                   **kwargs):
        self.curr_df = self.dfs[patient][[col1, col2]].copy()
        
        if do_scaling:
            self.curr_df[self.curr_df.columns] = scaler.fit_transform(self.curr_df)
        
        labels = self.fit_predict(patient=patient, col1=col1, col2=col2, do_scaling=do_scaling, **kwargs)

        if print_clust_num:
            print('Число кластеров:', len(set(labels)))
        
        scatterplot2d(self.curr_df if plot_scaled else self.dfs[patient], 
                      col1=col1, col2=col2, labels=labels, dots_size=dots_size)

Выбираем метод

In [ ]:
method_name = 'dbscan'
params_range = clustering[method_name]['params_range']

Создаем инстанс класса и прокидываем в него метод, границы на его параметры, данные и нормализатор (ощутил всю боль переводчиков)

In [ ]:
scaler = StandardScaler()  #MinMaxScaler()
clusterer = InteractiveClusterer(**clustering[method_name], 
                                 dfs=dfs, 
                                 scaler=scaler)

И, магия. Юпитеровский виждет: https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html

In [ ]:
interact(clusterer.analysis2d, 
         print_clust_num=True, 
         dots_size=[*range(1, 15)], 
         palette='coolwarm', 
         patient=[*range(0, 5)], 
         col1=cols, 
         col2=cols,
         do_scaling=[False, True], 
         plot_scaled=[False, True],
         **params_range)